In [1]:
# Standard library imports.
from datetime import datetime
from pathlib import Path
import sys

# Local imports.
sys.path.insert(0,str(Path.cwd().parent/"project_scripts"))
import retro_object as ro
import pandas as pd
import frame_functions as ff

# Create a Boston RetroObject.
boston = ro.Retro_Object("bos")

# Create a list of frames for the 2010 through 2020 seasons.
my_frames: list[pd.DataFrame] = []
for year in range(2010,2021):
    frame = boston.bevent(
        year = year,
        start = datetime(year,7,1),
        end = datetime(year,7,31)
        )
    my_frames.append(frame)

# Concatenate all of the DataFrames in my_frames.
boston_july_events = pd.concat(objs=my_frames)

# Strikeouts at Boston July home games, 2010 through 2020. 
strike_three = (
    boston_july_events
    .query(""" `event text*` == "K" """)
    .filter([
        "game id*", "date", "balls*", "strikes*",
        "visiting team*","batting team*",
        "res batter*", "res batter hand*",
        "res pitcher*", "res pitcher hand*"
        ])
    )

# Extract dates from game id* column.  Add batter and pitcher names to
# dataframe.
ff.get_bevent_dates(strike_three)
ff.insert_names(
    frame = strike_three,
    columns = ["res batter*","res pitcher*"]
    )

# Make minor replacements in the strike_three dataframe.
strike_three["batting team*"] = (
    strike_three["batting team*"]
    .replace({0: "away", 1: "home"})
    )

# Redirect csv file to our export directory.
strike_three.to_csv(
    Path.cwd().parent
    /"project_exports"
    /"july_strikeouts_third_export.csv",
    index = False
    )